# Metody obliczeniowe w nauce i technice

## Vadim Karpinskiy

### Laboratorium 1: Arytmetyka komputerowa

Programy, przedstawione w danym sprawozdaniu, są napisane w języku Python. Niestety, standardowa biblioteka Python nie obsługuje liczb pojedynczej precyzji. W celu dostarczenia takiej możliwości możemy zainstalować bibliotekę NumPy (Numerical Python), np. za pomocą następującej komendy:

In [ ]:
!pip install numpy --user

W celu tworzenia wykresów wykorzystamy bibliotekę Matplotlib:

In [ ]:
!pip install matplotlib --user

#### Zadanie 1: Sumowanie liczb pojedynczej precyzji

Poniższy program oblicza sumę N liczb pojedynczej precyzji przechowywanych w tablicy o rozmiarze N. Tablica wypełniona jest tą samą wartością v z przedziału [0.1, 0.9].

In [ ]:
import numpy as np

N = 10000000
v = np.float32(0.53125)
arr = [v]*N
sum_arr = [v]*N

def simple_sum():
    res = np.float32(0.0)
    for i in range(len(arr)):
        res += arr[i]
        sum_arr[i] = res
    return res

res = simple_sum()
print(res)

Wyznaczymy błędy obliczeń, korzystając ze wzorów, przedstawionych poniżej:

**Błąd bezwzględny**:
$$
\Delta{x} = |x - x_{0}|
$$ gdzie:   
$x$ - wartość dokładna,    
$𝑥_{0}$ - wartość obliczona

In [ ]:
absolute_error = abs(v*N - res)
print(absolute_error)

**Błąd względny**:
$$
\delta = \frac{\Delta{x}}{x} = \frac{|x - x_{0}|}{x}
$$ gdzie:     
$\Delta{x}$ - błąd bezwzględny obliczeń,    
$x$ - wartość dokładna,    
$𝑥_{0}$ - wartość obliczona

In [ ]:
relative_error = absolute_error / (v*N)
print(relative_error)

Wyrazimy błąd względny w procentach:
$$
\delta = \frac{\Delta{x}}{x} \cdot 100\% = 0.053018... \cdot 100\% \approx 5.301 \%
$$

Względny błąd obliczeń wynosi ponad 5% - jest zdecydowanie duży dla sekwencji prostych operacji arytmetycznych.
Wynika on przede wszystkim z kumulacji błędów w wyniku zaokrąglania. Zauważmy, że po każdej operacji sumowania zwiększa się wartość zmiennej, przechowującej wynik, natomiast wartość drugiego składnika nie zmienia się, co oznacza, że "odległość" między tymi liczbami rośnie. Dodawanie takich dwóch liczb zmiennoprzecinkowych będzie polegało na wyrównywaniu cech składników, a następnie normalizacji mantysy wyniku. Przy znacznej odłegłości mantysa nie zmieści się w zakresie, dlatego zostanie zaokrąglona, powodując utratę dokładności. 

Warto zauważyć, że na błąd obliczeń także wpływa błąd reprezentacji liczb w systemie komputerowym. 

#### Badanie zmiany błędu względnego obliczeń

Stworzymy wykres zależności wartości błędu względnego od liczby wykonanych operacji. Będziemy raportować wartość błędu co 25000 kroków:

In [ ]:
from matplotlib import pyplot as plt

def error_plot(arr,step):
    rel_err = []
    for i in range(1, N, step):
        tmp = abs(v*i - sum_arr[i-1])/(v*i)
        rel_err.append(tmp)
    return rel_err

plt.plot(error_plot(arr,25000))
plt.title("Wykres zależności wartości błędu względnego od liczby wykonanych operacji (1x25000)")
plt.show()

Przeanalizujemy powyższy wykres. Na początku błąd względny wynosi 0.00 - różnica między składnikami jest niewielka. Od pewnego k $\approx$ 40 normalizacja mantysy powoduje jej zaokrąglenie, dlatego pojawia się utrata precyzji. Ten błąd wpływa na każdą następną operację arytmetyczną, dlatego wykres rośnie tak gwałtownie.

#### Rekurencyjny algorytm sumowania

In [ ]:
def better_sum(arr, low, high):
    if low > high:
        return 0
    if low == high:
        return arr[low]
    mid = (low+high)//2
    return better_sum(arr,low,mid)+better_sum(arr,mid+1, high)

recc_sum = better_sum(arr,0,len(arr)-1)
print(recc_sum)

Wyznaczymy błędy obliczeń, korzystając ze wzorów (1) i (2):

In [ ]:
absolute_error = abs(v*N - recc_sum)
print(absolute_error)

In [ ]:
relative_error = absolute_error / (v*N)
print(relative_error)

W przypadku sumowania liczb za pomocą rekurencyjnego algorytmu błędy obliczeń wynoszą 0.0 - wynika to z tego, że wartości składników zwiększają się równomiernie (ostatni element tablicy może być mniejszy od pozostałych w przypadku nieparzystej długości tablicy).  

Oszacujemy czasy wykonania zwykłej i rekurencyjnej sumy:

In [ ]:
import time
start_time = time.time()
simple_sum()
print("Czas wykonania zwykłej sumy: ")
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import time
start_time = time.time()
better_sum(arr,0,len(arr)-1)
print("Czas wykonania rekurencyjnej sumy: ")
print("--- %s seconds ---" % (time.time() - start_time))

Zauważmy, że rekurencyjny algorytm też może zwracać niezerowy błąd. Przykładowo, dla następującej tablicy:

In [ ]:
example = [0]*10
eps = np.float32(0.00001)
for i in range(10):
    example[i] = eps*pow(100,i)
print(example)

acc = np.float32(0.0)
for i in range(10):
    acc += example[i]

Błąd bezwzględny wynosi:

In [ ]:
absolute_error = abs(10101010101010.101010 - acc)
print(absolute_error)

### Zadanie 2: Algorytm Kahana

In [ ]:
def kahan_algorithm(arr):
    res = np.float32(0.0)
    err = np.float32(0.0)

    for i in range(len(arr)):
        y = arr[i] - err
        temp = res + y
        err = (temp - res) - y
        res = temp
    return res

kahan_res = kahan_algorithm(arr)
print(kahan_res)

Błąd bezwzględny:

In [ ]:
absolute_error = abs(v*N - kahan_res)
print(absolute_error)

In [ ]:
relative_error = absolute_error / (v*N)
print(relative_error)

Zmienna **err** służy do odejmowania błędu od następnej liczby. Składa się ona z dwóch części: (temp - res) naprawia wyższe bity y, a (-y) naprawia niższe bity y.

Oszacujemy czas wykonania algorytmu Williama Kahana:

In [ ]:
import time
start_time = time.time()
kahan_algorithm(arr)
print("Czas wykonania algorytmu Kahana: ")
print("--- %s seconds ---" % (time.time() - start_time))

Algorytm Kahana działa znacznie szybciej, niż algorytm rekurencyjnego sumowania.

### Zadanie 3: Sumy częściowe

Rozważmy sumy częściowe szeregu definiującego funkcję dzeta Riemanna:
    $$
    \zeta(s) = \sum_{k=1}^n \frac{1}{k^{s}}
    $$

oraz funkcję eta Dirichleta
    $$
    \eta(s) = \sum_{k=1}^n {(-1)}^{k-1}\frac{1}{k^{s}}
    $$

Poniższe funkcje służą do obliczania wartości funkcji $\zeta(s)$ oraz $\eta(s)$ dwoma sposobami: w przód i w tył:

In [ ]:
def dzeta_forward(result, n, s):
    for k in range(1, n):
        result += 1 / pow(k,s)
    return result


def dzeta_backward(result, n, s):
    for k in range(n, 0, -1):
        result += 1 / pow(k,s)
    return result


def eta_forward(result, n, s):
    for k in range(1, n):
        result += pow((-1),(k-1)) / pow(k,s)
    return result


def eta_backward(result, n, s):
    for k in range(n, 0, -1):
        result += pow((-1),(k-1)) / pow(k,s)
                                     
    return result

Tworzymy 3 tablice:
 - **single_prec**: służy do przechowywania liczb pojedynczej precyzji
 - **double_prec**: służy do przechowywania liczb podwójnej precyzji
 -  **series_upperbound**: służy do przechowywania liczby iteracji

In [ ]:
single_prec = [np.float32(2.0), np.float32(3.6667), np.float32(5.0), np.float32(7.2), np.float32(10.0)]
double_prec = [2.0, 3.6667, 5.0, 7.2, 10.0]
series_upperbound = [50, 100, 200, 500, 1000]

In [ ]:
def dzeta():
    print("DZETA")
    print("------------")
    for s in range(len(single_prec)):
        print(single_prec[s])
        print("-----------------")
        for n in range(len(series_upperbound)):
            
            print("Dzeta 1->n: ")
            print(dzeta_forward(np.float32(0.0),   series_upperbound[n],    single_prec[s]), end= " ")
            print(dzeta_forward(0.0,            series_upperbound[n],   double_prec[s]))

            print("Dzeta n->1: ")
            print(dzeta_backward(np.float32(0.0),series_upperbound[n],    single_prec[s]), end = " ")
            print(dzeta_backward(0.0,series_upperbound[n],   double_prec[s]))
        print("------------------")
dzeta()

In [ ]:
def eta():
    print("ETA")
    print("------------")
    for s in range(len(single_prec)):
        print(single_prec[s])
        print("-----------------")
        for n in range(len(series_upperbound)):
            
            print("Eta 1->n: ")
            print(eta_forward(np.float32(0.0),   series_upperbound[n],    single_prec[s]), end= " ")
            print(eta_forward(0.0,            series_upperbound[n],   double_prec[s]))

            print("Eta n->1: ")
            print(eta_backward(np.float32(0.0),series_upperbound[n],    single_prec[s]), end = " ")
            print(eta_backward(0.0,series_upperbound[n],   double_prec[s]))
        print("------------------")
eta()

Analizując wyniki, możemy stwierdzić, że operacje arytmetyczne nie są łączne w architekturze komputerowej. Sumując w przód mamy przypadek podobny do zadania 1: do większej liczby liczby coraz mniejsze, co powoduje zaokrąglenie mantysy i utratę dokładności. Sumując w tył do bardzo małej liczby dodajemy wartości coraz większe, zmniejszając "odległość" pomiędzy składnikami.

W przypadku funkcji eta mamy odwrotną sytuację: chcemy uniknąć odejmowania bliskich liczb, bo wynikiem będzie mała liczba, mająca dużo zer tam, gdzie jej "poprzednicy" mieli cyfry znaczące. Normalizowanie takiej liczby spowoduje tzw. *catastrophic cancellation*